In [16]:
import openpyxl
from openpyxl import Workbook
from copy import deepcopy
from typing import Iterable

In [17]:
def findNotNone(row):
    """엑셀의 한행 데이터 중, None이 아닌 항목 찾기"""
    return list(filter(lambda x: x != None, row))

def countNotNone(vals):
    """엑셀의 한행 데이터 중, None이 아닌 항목 개수 세기"""
    return list(map(len, (map(findNotNone, vals))))

In [18]:
help(countNotNone)

Help on function countNotNone in module __main__:

countNotNone(vals)
    엑셀의 한행 데이터 중, None이 아닌 항목 개수 세기



In [20]:
def make_sequential_group(nums):
    """리스트 중 연속한 숫자들을 서브리스트로 구성한다"""
    res = []
    tmp = []
    enum_list = list(enumerate(nums))
    start_i,start_e = enum_list[0]
    for i,e in enum_list:
        if (e - start_e) == (i-start_i):
            if i == len(nums)-1:
                tmp.append(e)
                res.append(deepcopy(tmp))
            else:
                tmp.append(e)
                
        elif (e - start_e) != (i-start_i):
            if i == len(nums)-1:
                res.append(deepcopy(tmp))
                tmp.clear()
                tmp.append(e)
                res.append(deepcopy(tmp))
            else:
                res.append(deepcopy(tmp))
                tmp.clear()
                start_i,start_e = i,e
                tmp.append(e)
        else:
            res.append(deepcopy(tmp))

    return res

In [44]:
def find_MergedArea(rowObjs, colObj):
    """엑셀칼럼객체에서 병합된 셀의 영역idx를 찾는다"""
    startRow_idx = find_header_startRow_idx(rowObjs)
    all_idx_merged =  list(filter(lambda x: x > startRow_idx,
                             (map(lambda x: x[0],
                                  (filter(lambda x: "Merged" in str(x),
                                          enumerate(colObj)))))))
    all_idx_merged.insert(0, startRow_idx)
    sequential_group = make_sequential_group(all_idx_merged)

    return sequential_group

In [48]:
def find_header_startRow_idx(rowObjs):
    """None이 아닌 유효 데이터가 갑자기 많아지는 Row idx 찾기(헤더의 시작)"""
    vals = [[x.value for x in y] for y in rowObjs]
    validData_count = countNotNone(vals)
    
    for i,j in enumerate(validData_count):
        if j - (validData_count[i-1]) >= 5:
            return i

In [49]:
def find_header_endRow_idx(rowObjs, colObjs, startRow_idx):
    """헤더의 끝 row idx 찾기"""
    MergedAreas = list(map(lambda x: find_MergedArea(rowObjs,x), colObjs))
    firstElems_MaybeHeader = map(lambda x: x[0], MergedAreas)
    last_idxs = map(lambda x: x[-1], firstElems_MaybeHeader)
    
    return max(last_idxs)

In [174]:
def fill_missing_h1(headers):
    """find_header_objs 함수 호출 시, 헤더 최상위 레벨의 병합부분 누락 데이터 채움용"""
    new_h1 = deepcopy(headers[0])
    for i,v in enumerate(headers[0]):
        if v == None:
            new_h1[i] = new_h1[i-1]
    headers[0] = new_h1
    return headers

def find_header_objs(rowObjs, colObjs):
    """헤더 startRow_idx와 endRow_idx를 기반으로 헤더객체들 추출 """
    startRow_idx = find_header_startRow_idx(rowObjs)
    endRow_idx = find_header_endRow_idx(rowObjs, colObjs, startRow_idx)
    range_ = range(startRow_idx, endRow_idx+1)
    find_header = lambda rowObj: map(lambda x: x.value, rowObj)
    all_headers = list(map(lambda x: list(find_header(rowObjs[x])), range_))
    all_headers_h1_filled = fill_missing_h1(all_headers)
    zip_all_headers = zip(*all_headers_h1_filled)    
    
    return zip_all_headers

In [175]:
def find_targetColumn_idx(rowObjs, colObjs, _target):
    """타겟 문자열이 포함된 헤더가 위치한 칼럼 인덱스 추출"""
    headers = list(find_header_objs(rowObjs, colObjs))
    headers_with_idx = list(enumerate(headers))
    idx = list(map(lambda x: x[0], headers_with_idx))
    str_headers = list(map(lambda x: str(x[1]), headers_with_idx))
    str_headers_with_idx = list(zip(idx, str_headers))
    
    targets = _target.split()
    
    def is_all_include(targets, strs):
        res = []
        for i in targets:
            if i in strs:  res.append(True)
            else        :  res.append(False)
        return all(res)
    
    find_f = lambda y: list(filter(lambda x: is_all_include(y ,x[1]), str_headers_with_idx))
    
    out = find_f(targets)
    
    return out

---
<br>

---

In [54]:
## 검색할 파일명 기입 ##
파일명 = "Building List Resource_20230110.xlsx"

In [31]:
def set_search_engine(filename, header_identifier):
    doc = openpyxl.load_workbook(f'{파일명}')
    
    return doc

In [265]:
search_engine = set_search_engine(파일명, "Name")
search_engine.sheetnames

['EGAT',
 'HDO RVMP 2020TA',
 'HDO RVMP',
 'HDO SDA',
 'Vistok',
 'IKAN',
 'UTCC',
 'KLNG',
 'SGPC',
 'TACE',
 'HPP4',
 'PTT LNG',
 'LINE',
 'HNCC PKG-1',
 'HNCC PKG-2',
 'HNCC PKG-3',
 'P-GMTP3',
 'MMCC',
 'IKCP',
 'TVEP']

In [266]:
sheet = search_engine["TACE"]  # 임시로  search_engine이 doc 역할( 테스트 후 리팩토링 시 주석 삭제 예정)
sheet

<Worksheet "TACE">

In [267]:
def seach_in_targetSheet(sheet, search_target, target_prop):
    sheet_name = sheet.title
    lookup_target = "Building Name"  ##테스트 후 엔진객체에서 추출하는 걸로 수정
    rowObjs = list(sheet.rows)
    colObjs = list(sheet.columns)
    head_startRow_idx = find_header_startRow_idx(rowObjs)
    head_endRow_idx = find_header_endRow_idx(rowObjs, colObjs, head_startRow_idx)
    lookup_target_col_idx = find_targetColumn_idx(rowObjs, colObjs, lookup_target)[0][0]
    
    f = lambda y: list(map(lambda x: x.value, colObjs[lookup_target_col_idx]))[y+1:]
    data_NoneStr = list(map(str, f(head_endRow_idx)))
    all_target_data = list(filter(lambda x: x[1] != 'None', enumerate(data_NoneStr)))
    
    search_target_data = list(filter(lambda x: search_target in x[1], all_target_data))
    search_target_row_idxs = list(map(lambda x: x[0]+head_endRow_idx+1,search_target_data))
    search_target_col_idxs = find_targetColumn_idx(rowObjs, colObjs, target_prop)
    search_target_col_idx = search_target_col_idxs[0][0]  ## 임시로 첫째항목 기준 데이터 추출로 구현함(추후 수정예정)
    
    def find_target_prop_data(rowObjs, row_idx, col_idx):
        return list(map(lambda x: x.value, rowObjs[row_idx]))[col_idx]
    
    goal_data = list(map(lambda x: find_target_prop_data(rowObjs, x, search_target_col_idx), search_target_row_idxs))
    goal_data_identi = list(map(lambda x: x[1] ,search_target_data))
    goal_data_pair = list(zip(goal_data_identi, goal_data))
    
    result_dict = {sheet_name: goal_data_pair}
    
    return result_dict

In [271]:
seach_in_targetSheet(sheet, "Shelter", "Area")

{'TACE': [(' Co-Catalyst Shelter', '=22.8*6.5'),
  (' Communication center &\n Air Raid Shelter & Training Building',
   '=55.55*14'),
  (' Sea water Chlorination Shelter', '=7.1*8'),
  (' Sea water Filtration Shelter', '=8.6*5.4')]}